In [1]:
spark

Intitializing Scala interpreter ...

Spark Web UI available at http://192.168.0.7:4040
SparkContext available as 'sc' (version = 3.1.2, master = local[*], app id = local-1642495560845)
SparkSession available as 'spark'


res0: org.apache.spark.sql.SparkSession = org.apache.spark.sql.SparkSession@21d49f03


이 장에서는 스파크에서 기본으로 지원하는 데이터 소스뿐만 아니라 커뮤니티에서 만들어낸 수많은 데이터 소스를 소개함<br/>
스파크에는 6가지 핵심 데이터 소스와 커뮤니티에서 만든 수백 가지 외부 데이터 소스가 있음<br/>
스파크의 핵심 데이터 소스<br/>
* CSV
* JSON
* 파케이
* ORC
* JDBC/ODBC 연결
* 일반 텍스트 파일

앞서 언급했듯 스파크에는 커뮤니티에서 만든 수많은 데이터 소스가 존재하며, 그중 일부는 다음과 같음<br/>
* 카산드라
* HBase
* MongoDB
* AWS RedShift
* XML
* 기타 수많은 데이터 소스
이 장의 목표는 **스파크의 핵심 데이터 소스를 이용해 데이터를 읽고 쓰는 방법을 터득하고, third party 데이터 소스와 스파크를 연동할 때 무엇을 고려해야 하는지 배우는 것**임<br/>
이 목표를 이루기 위해 반드시 이해해야 하는 핵심 개념을 먼저 살펴보겠음<br/>

# 9.1 데이터 소스 API의 구조
특정 포맷을 읽고 쓰는 방법을 알아보기 전에 데이터 소스 API의 전체적인 구조를 알아보겠음<br/>

## 9.1.1 읽기 API의 구조
데이터 읽기의 핵심 구조는 다음과 같음<br/>
*DataFrameReader.format(...).option("key", "value").schema(...).load()*

모든 데이터 소스를 읽을 때 위와 같은 형식을 사용함<br/>
format 메서드는 선택적으로 사용할 수 있으며, 기본값은 파케이 포맷임<br/>
그리고 option 메서드를 사용해 데이터를 읽는 방법에 대한 파라미터를 키-값 쌍으로 설정할 수 있음<br/>
마지막으로 schema 메서드는 데이터 소스에서 스키마를 제공하거나, 스키마 추론 기능을 사용하려는 경우에 선택적으로 사용할 수 있음<br/>
당연히 데이터 포맷별로 필요한 몇 가지 옵션이 존재함<br/>
이 부분은 개별 데이터 포맷을 다룰 때 알아보겠음<br/>

## 9.1.2 데이터 읽기의 시초
스파크에서 데이터를 읽을 때는 기본적으로 DataFrameReader를 사용함<br/>
DataFrameReader는 SparkSession의 read 속성으로 접근함<br/>

In [2]:
spark.read

res1: org.apache.spark.sql.DataFrameReader = org.apache.spark.sql.DataFrameReader@6edde366


DataFrameRedaer를 얻은 다음에는 다음과 같은 값을 지정해야 함<br/>
* 포맷
* 스키마 
* 읽기 모드
* 옵션

포맷, 스키마, 그리고 옵션은 transformation을 추가로 정의할 수 있는 DataFrameReader를 반환함<br/>
그리고 읽기 모드는 제외한 3가지 항목은 필요한 경우에만 선택적으로 지정할 수 있음<br/>
데이터 소스마다 데이터를 읽는 방식을 결정할 수 있는 옵션을 제공함<br/>
사용자는 DataFrameReader에 반드시 데이터를 읽을 경로를 지정해야 함<br/>
전반적인 코드 구성은 다음과 같음<br/>

In [3]:
/*
spark.read.format("csv")
    .option("mode", "FAILFAST")
    .option("inferSchema", "true")
    .option("path", "path/to/file(s)")
    .schema(someSchema)
    .load()
*/

옵션을 설정할 수 있는 다양한 방법이 존재함<br/>
예를 들어 설정값을 가진 맵 객체를 전달할 수 있음<br/>
하지만 당분간은 위 예제와 같은 방법을 사용하겠음<br/>

### 읽기 모드

외부 데이터소스에서 데이터를 읽다 보면 형식에 맞지 않는 데이터를 만나게 됨<br/>
특히 반정형 데이터 소스를 다룰 때 이런 경우가 많이 발생함<br/>
읽기 모드는 스파크가 형식에 맞지 않는 데이터를 만났을 때의 동작 방식을 지정하는 옵션임<br/>
(읽기 모드는 기본값은 permissive임)

## 9.1.3 쓰기 API 구조

데이터 쓰기의 핵심 기초는 다음과 같음<br/>

In [4]:
/*
DataFrameWriter.format(...).option(...).partitionBy(...).bucketBy(...).sortBy(...).save()
*/

모든 데이터 소스에 데이터를 쓸 때 위와 같은 형식을 사용함<br/>
format 메서드는 선택적으로 사용할 수 있으며 기본값은 파케이 포맷임<br/>
그리고 option 메서드를 사용해 데이터 쓰기 방법을 설정할 수 있음<br/>
**partitionBy, bucketBy, sortBy 메서드는 파일 기반의 데이터 소스에서만 동작하며, 이 기능으로 최종 파일 배치 형태(layout)를 제어할 수 있음**<br/>

## 9.1.4 데이터 쓰기의 기초

데이터 쓰기는 데이터 읽기와 매우 유사하며, DataFrameReader 대신 DataFrameWriter를 사용함<br/>
데이터 소스에 항상 데이터를 기록해야 하기 때문에 DataFrame의 write 속성을 이용해 DataFrame별로 DataFrameWriter에 접근해야 함<br/>

In [5]:
/*
dataFrame.write
*/

DataFrameWriter를 얻은 다음에는 포맷(format), 옵션(option), 저장(save) 모드를 지정해야 하며, 데이터가 저장될 경로를 반드시 입력해야 함<br/>
데이터 소스별로 다양한 옵션이 존재하며 개별 데이터 소스를 다룰 때 자세히 알아보겠음<br/>

In [6]:
/*
dataframe.write.format("csv")
    .option("mode", "OVERWRITE")
    .option("dateFormat", "yyyy-mm-dd")
    .option("path", "path/to/file(s)")
    .save()
*/

### 저장 모드

저장 모드는 스파크가 지정된 위치에서 동일한 파일을 발견했을 때의 동작 방식을 지정하는 옵션임<br/>
스파크의 저장 모드<br/>
* append: 해당 경로에 이미 존재하는 파일 목록에 결과 파일을 추가함
* overwrite: 이미 존재하는 모든 데이터를 완전히 덮어씀
* errorIfExists: 해당 경로에 데이터나 파일이 존재하는 경우 오류를 발생시키면서 쓰기 작업이 실패함
* ignore: 해당 경로에 데이터나 파일이 존재하는 경우 아무런 처리도 하지 않음

기본값은 errorIfExists임<br/>

지금까지 데이터 소스를 사용할 때 필요한 핵심 개념을 대부분 알아보았음<br/>
이제부터 스파크의 핵심 데이터 소스에 대해 알아보겠음<br/>

# 9.2 CSV 파일

CSV(comma-separated-values)는 콤마(,)로 구분된 값을 의미함<br/>
CSV는 각 줄이 단일 레코드가 되며 레코드의 각 필드를 콤마로 구분하는 일반적인 텍스트 파일 포맷임<br/>
CSV 파일은 구조적으로 보이지만, 사실 매우 까다로운 파일 포맷 중 하나임<br/>
그 이유는 운영 환경에서는 어떤 내용이 들어 있는지, 어떠한 구조로 되어 있는지 등 다양한 전제를 만들어낼 수 없기 때문임<br/>
그러한 이유로 CSV reader는 많은 수의 옵션을 제공함<br/>
예를 들어 CSV 파일 내 컬럼 내용에 콤마가 들어 있거나 비표준적인 방식으로 null 값이 기록된 경우 특정 문자를 escape 처리하는 옵션을 사용해 문제를 해결할 수 있음<br/>

## 9.2.1 CSV 옵션

(책에 있는 표를 참고할 것)

## 9.2.2 CSV 파일 읽기

다른 포맷과 마찬가지로 CSV 파일을 읽으려면 먼저 CSV용 DataFrameReader를 생성해야 하며 예제는 다음과 같음<br/>

In [7]:
spark.read.format("csv")

res6: org.apache.spark.sql.DataFrameReader = org.apache.spark.sql.DataFrameReader@7a47c7bb


그 다음에는 스키마와 읽기 모드를 지정함<br/>
이제 몇 가지 옵션을 지정해보겠음<br/>
header 옵션은 CSV 파일을 읽기 위해 true로, mode 옵션은 FAILFAST로, inferSchema 옵션은 true로 설정함<br/>

In [8]:
/*
spark.read.format("csv")
    .option("header", "true")
    .option("mode", "FAILFAST")
    .option("inferSchema", "true")
    .load("some/path/to/file.csv")
*/

앞서 언급했듯 비정상적인 데이터를 얼마나 수용할 수 있을지 읽기 모드로 지정할 수 있음<br/>
예를 들어 다음과 같이 읽기 모드와 5장에서 생성한 스키마를 파일의 데이터가 예상한 형태로 이루어져 있음을 검증하는 용도로 사용할 수 있음<br/>

In [9]:
import org.apache.spark.sql.types.{StructField, StructType, StringType, LongType}

val myManualSchema = new StructType(Array(
    new StructField("DEST_COUNTRY_NAME", StringType, true),
    new StructField("ORIGIN_COUNTRY_NAME", StringType, true),
    new StructField("count", LongType, false)
))

spark.read.format("csv")
    .option("header", "true")
    .option("mode", "FAILFAST")
    .schema(myManualSchema)
    .load("Downloads/Spark-The-Definitive-Guide/data/flight-data/csv/2010-summary.csv")
    .show(10)

+-----------------+-------------------+-----+
|DEST_COUNTRY_NAME|ORIGIN_COUNTRY_NAME|count|
+-----------------+-------------------+-----+
|    United States|            Romania|    1|
|    United States|            Ireland|  264|
|    United States|              India|   69|
|            Egypt|      United States|   24|
|Equatorial Guinea|      United States|    1|
|    United States|          Singapore|   25|
|    United States|            Grenada|   54|
|       Costa Rica|      United States|  477|
|          Senegal|      United States|   29|
|    United States|   Marshall Islands|   44|
+-----------------+-------------------+-----+
only showing top 10 rows



import org.apache.spark.sql.types.{StructField, StructType, StringType, LongType}
myManualSchema: org.apache.spark.sql.types.StructType = StructType(StructField(DEST_COUNTRY_NAME,StringType,true), StructField(ORIGIN_COUNTRY_NAME,StringType,true), StructField(count,LongType,false))


위 코드를 잘 동작하지만, 데이터가 기대했던 데이터 포맷이 아니었다면 문제가 발생했을 것<br/>
예를 들어 현재 스키마의 모든 컬럼을 LongType으로 변경해보자<br/>
**실제** 스키마와 일치하지 않지만, 스파크는 어떤 문제도 찾아내지 못함<br/>
문제는 스파크가 실제로 데이터를 읽어들이는 시점에 발생함<br/>
데이터가 지정된 스키마에 일치하지 않으므로 스파크 job은 시작하자마자 종료됨<br/>

In [10]:
// 실제 스키마와 일치하지 않음

val myManualSchema = new StructType(Array(
    new StructField("DEST_COUNTRY_NAME", LongType, true),
    new StructField("ORIGIN_COUNTRY_NAME", LongType, true),
    new StructField("count", LongType, false)
))

spark.read.format("csv")
    .option("header", "true")
    .option("mode", "FAILFAST")
    .schema(myManualSchema)
    .load("Downloads/Spark-The-Definitive-Guide/data/flight-data/csv/2010-summary.csv")
    .take(10)

org.apache.spark.SparkException:  Job aborted due to stage failure: Task 0 in stage 1.0 failed 1 times, most recent failure: Lost task 0.0 in stage 1.0 (TID 1) (192.168.0.7 executor driver): org.apache.spark.SparkException: Malformed records are detected in record parsing. Parse Mode: FAILFAST. To process malformed records as null result, try setting the option 'mode' as 'PERMISSIVE'.

스파크는 **지연 연산** 특성이 있으므로 DataFrame 정의 시점이 아닌 job 실행 시점에만 오류가 발생함<br/>
예를 들어 DataFrame을 정의하는 시점에는 존재하지 않는 파일을 지정해도 오류가 발생하지 않음<br/>

*의문 사항*<br/>
위에 보면<br/>
org.apache.spark.SparkException: Job aborted due to stage failure: Task 0 in stage 1.0 failed 1 times, most recent failure: Lost task 0.0 in stage 1.0 (TID 1) (192.168.0.7 executor driver): org.apache.spark.SparkException: Malformed records are detected in record parsing.<br/>
이런 메시지가 뜨는데 이건 오류가 발생한 것으로 보지 않은 건지?

## 9.2.3 CSV 파일 쓰기

데이터 읽기와 마찬가지로 CSV 파일을 쓸 때 사용할 수 있는 다양한 옵션이 있음<br/>
maxColumns와 inferSchema 옵션 같이 데이터 쓰기에는 적용되지 않는 옵션을 제외하면 데이터 읽기와 동일한 옵션을 제공함<br/>
코드 예제는 다음과 같음<br/>

In [11]:
val csvFile = spark.read.format("csv")
    .option("header", "true")
    .option("mode", "FAILFAST")
    .option("inferSchema", "true")
    .load("Downloads/Spark-The-Definitive-Guide/data/flight-data/csv/2010-summary.csv")

csvFile: org.apache.spark.sql.DataFrame = [DEST_COUNTRY_NAME: string, ORIGIN_COUNTRY_NAME: string ... 1 more field]


# 9.3 JSON 파일

자바스크립트 세상에서 온 파일 형식들은 자바스크립트 객체 표기법, 즉 JSON(JavaScript Object Notation)으로 더 친숙하게 알려져 있음<br/>
이런 종류의 데이터를 다룰 때 조심해야 하는 사항을 먼저 알아보겠음<br/>
스파크에서는 JSON 파일을 사용할 때 **줄로 구분된** JSON을 기본적으로 사용함<br/>
이런 방식은 큰 JSON 객체나 배열을 하나씩 가지고 있는 파일을 다루는 것과는 대조적인 부분임<br/>

*multiLine* 옵션을 사용해 줄로 구분된 방식과 여러 줄로 구성된 방식을 선택적으로 사용할 수 있음<br/>
이 옵션을 true로 설정하면 전체 파일을 하나의 JSON 객체로 읽을 수 있음<br/>
스파크는 JSON 파일을 파싱한 다음에 DataFrame을 생성함<br/>
**줄로 구분된 JSON은 전체 파일을 읽어 들인 다음 저장하는 방식이 아니므로 새로운 레코드를 추가할 수 있음**<br/>
다른 포맷에 비해 훨씬 더 안정적인 포맷이므로 이 방식을 사용하는 것이 좋음<br/>
줄로 구분된 JSON이 인기 있는 또 다른 이유는 구조화되어 있고, 최소한의 기본 데이터 타입이 존재하기 때문임(JSON은 자바스크립트를 토대로 만들어졌음)<br/>
따라서 스파크는 적합한 데이터 타입을 추정할 수 있어 원활하게 처리할 수 있음<br/>
JSON은 객체이기 때문에 CSV보다 옵션 수가 적음<br/>

## 9.3.1 JSON 옵션

(책에 있는 표를 참고할 것)

## 9.3.2 JSON 파일 읽기

JSON 파일을 읽는 방법과 앞에서 살펴본 옵션을 비교해볼 수 있는 예제를 살펴보겠음<br/>

In [15]:
// 앞에서 일부러 잘못 설정한 스키마 되돌려놓기
import org.apache.spark.sql.types.{StructField, StructType, StringType, LongType}
import org.apache.spark.sql.types.Metadata

val myManualSchema = new StructType(Array(
    new StructField("DEST_COUNTRY_NAME", StringType, true),
    new StructField("ORIGIN_COUNTRY_NAME", StringType, true),
    new StructField("count", LongType, false,
                   Metadata.fromJson("{\"hello\":\"world\"}"))
))

import org.apache.spark.sql.types.{StructField, StructType, StringType, LongType}
import org.apache.spark.sql.types.Metadata
myManualSchema: org.apache.spark.sql.types.StructType = StructType(StructField(DEST_COUNTRY_NAME,StringType,true), StructField(ORIGIN_COUNTRY_NAME,StringType,true), StructField(count,LongType,false))


In [19]:
spark.read.format("json").load("Downloads/Spark-The-Definitive-Guide/data/flight-data/json/2010-summary.json").schema

res14: org.apache.spark.sql.types.StructType = StructType(StructField(DEST_COUNTRY_NAME,StringType,true), StructField(ORIGIN_COUNTRY_NAME,StringType,true), StructField(count,LongType,true))


In [21]:
spark.read.format("json").option("mode", "FAILFAST").
    load("Downloads/Spark-The-Definitive-Guide/data/flight-data/json/2010-summary.json").show(10)

+-----------------+-------------------+-----+
|DEST_COUNTRY_NAME|ORIGIN_COUNTRY_NAME|count|
+-----------------+-------------------+-----+
|    United States|            Romania|    1|
|    United States|            Ireland|  264|
|    United States|              India|   69|
|            Egypt|      United States|   24|
|Equatorial Guinea|      United States|    1|
|    United States|          Singapore|   25|
|    United States|            Grenada|   54|
|       Costa Rica|      United States|  477|
|          Senegal|      United States|   29|
|    United States|   Marshall Islands|   44|
+-----------------+-------------------+-----+
only showing top 10 rows



## 9.3.3 JSON 파일 쓰기

JSON 파일 쓰기는 읽기와 마찬가지로 간단하고 데이터 소스에 관계없이 JSON 파일에 저장할 수 있음 <br/>
그러므로 이전에 만들었던 CSV DataFrame을 JSON 파일의 소스로 재사용할 수 있음<br/>
이 작업 역시 이전 규칙을 그대로 따름<br/>
파티션당 하나의 파일을 만들며 전체 DataFrame을 단일 폴더에 저장함<br/>
JSON 객체는 한 줄에 하나씩 기록됨<br/>

In [23]:
/*
csvFile.write.format("json").mode("overwrite").save("workspace/tmp/my-json-file.json")
*/

# 9.4 파케이 파일

파케이는 다양한 스토리지 최적화 기술을 제공하는 오픈소스로 만들어진 *컬럼 기반의 데이터 저장 방식*임<br/>
특히 분석 워크로드에 최적화되어 있음<br/>
*저장소 공간을 절약할 수 있고 전체 파일을 읽는 대신 개별 파일을 읽을 수 있으며, 컬럼 기반의 압축 기능도 제공*함<br/>
**특히 스파크와 잘 호환되기 때문에 스파크의 기본 파일 포맷이기도 함**<br/>
파케이 파일은 읽기 연산 시 JSON이나 CSV보다 훨씬 효율적으로 동작하므로 장기 저장용 데이터는 파케이 포맷으로 저장하는 것이 좋음<br/>
파케이의 또 다른 장점은 복합 데이터 타입을 지원한다는 것임<br/>
컬럼이 배열, 맵, 구조체 데이터 타입이라 해도 문제없이 읽고 쓸 수 있음<br/>
단, CSV에서는 배열을 사용할 수 없음<br/>
파케이를 읽기 포맷으로 지정하는 방법은 다음과 같음<br/>

In [24]:
spark.read.format("parquet")

res19: org.apache.spark.sql.DataFrameReader = org.apache.spark.sql.DataFrameReader@24a5ecdf


## 9.4.1 파케이 파일 읽기

파케이는 옵션이 거의 없음<br/>
데이터를 저장할 때 자체 스키마를 사용해 데이터를 저장하기 때문<br/>
따라서 포맷을 설정하는 것만으로도 충분함<br/>
DataFrame을 표현하기 위해 정확한 스키마가 필요한 경우에만 스키마를 설정함<br/>
하지만 이런 작업은 거의 필요 없음<br/>
그 이유는 CSV 파일에서 inferSchema를 사용하는 것과 유사하게 읽는 시점에 스키마를 알 수 있기 때문(스키마 온 리드)<br/>
**파케이 파일은 스키마가 파일 자체에 내장되어 있으므로 추정이 필요 없음**<br/>
그러므로 이 방법이 더 효과적임<br/>

In [25]:
spark.read.format("parquet")
    .load("Downloads/Spark-The-Definitive-Guide/data/flight-data/parquet/2010-summary.parquet").show(10)

+-----------------+-------------------+-----+
|DEST_COUNTRY_NAME|ORIGIN_COUNTRY_NAME|count|
+-----------------+-------------------+-----+
|    United States|            Romania|    1|
|    United States|            Ireland|  264|
|    United States|              India|   69|
|            Egypt|      United States|   24|
|Equatorial Guinea|      United States|    1|
|    United States|          Singapore|   25|
|    United States|            Grenada|   54|
|       Costa Rica|      United States|  477|
|          Senegal|      United States|   29|
|    United States|   Marshall Islands|   44|
+-----------------+-------------------+-----+
only showing top 10 rows



### 파케이 옵션

(책에 있는 표를 참고할 것)

## 9.4.2 파케이 파일 쓰기

파케이 파일 쓰기는 읽기만큼 쉽고 파일의 경로만 명시하면 됨<br/>
분할 규칙은 다른 포맷과 동일하게 적용됨<br/>

In [26]:
/*
csvFile.write.format("parquet").mode("overwrite")
    .save("workspace/tmp/my-parquet-file.parquet")
*/

# 9.5 ORC 파일

ORC는 하둡 워크로드를 위해 설계된 자기 기술적(self-describing)이며 데이터 타입을 인식할 수 있는 컬럼 기반의 파일 포맷임<br/>
이 포맷은 대규모 스트리밍 읽기에 최적화되어 있을 뿐만 아니라 필요한 로우를 신속하게 찾아낼 수 있는 기능이 통합되어 있음<br/>
스파크는 ORC 파일 포맷을 효율적으로 사용할 수 있으므로 별도의 옵션 지정 없이 데이터를 읽을 수 있음<br/>

*ORC와 파케이의 차이점은?*<br/>
두 포맷은 매우 유사하나 근본적인 차이점이 있음<br/>
파케이는 스파크에 최적화된 반면 ORC는 하이브에 최적화되어 있음<br/>

## 9.5.1 ORC 파일 읽기

스파크에서 ORC 파일을 읽는 방법은 다음과 같음<br/>

In [27]:
spark.read.format("orc").load("Downloads/Spark-The-Definitive-Guide/data/flight-data/orc/2010-summary.orc").show(10)

+-----------------+-------------------+-----+
|DEST_COUNTRY_NAME|ORIGIN_COUNTRY_NAME|count|
+-----------------+-------------------+-----+
|    United States|            Romania|    1|
|    United States|            Ireland|  264|
|    United States|              India|   69|
|            Egypt|      United States|   24|
|Equatorial Guinea|      United States|    1|
|    United States|          Singapore|   25|
|    United States|            Grenada|   54|
|       Costa Rica|      United States|  477|
|          Senegal|      United States|   29|
|    United States|   Marshall Islands|   44|
+-----------------+-------------------+-----+
only showing top 10 rows



## 9.5.2 ORC 파일 쓰기

지금까지 본 것과 동일한 사용 패턴을 따르며 포맷을 지정한 다음 파일을 저장함<br/>

In [28]:
/*
csvFile.write.format("orc").mode("overwrite").save("/tmp/my-orc-file.orc")
*/

# 9.6 SQL 데이터베이스

SQL 데이터 소스는 매우 강력한 커넥터 중 하나임<br/>
사용자는 SQL을 지원하는 다양한 시스템에 SQL 데이터 소스를 연결할 수 있음<br/>
예를 들어 MySQL, PostgreSQL, Oracle 데이터베이스에 접속할 수 있음<br/>
또한 예제에서 사용할 SQLite에도 접속할 수 있음<br/>
데이터베이스는 원시 파일 형태가 아니므로 고려해야 할 옵션이 더 많음<br/>
예를 들어 데이터베이스의 인증 정보나 접속과 관련된 옵션이 필요함<br/>
그리고 스파크 클러스터에서 데이터베이스 시스템에 접속 가능한지 네트워크 상태를 확인해야 함<br/>

데이터베이스를 설정하는 번거로움을 없애고 이 책의 목적에 충실하기 위해 SQLite 실행을 위한 참고용 샘플을 제공함<br/>
모든 예제는 분산 환경이 아닌 로컬 머신에서도 충분히 테스트할 수 있어야 함<br/>
그러므로 로컬 머신에 간단하게 설치할 수 있는 SQLite를 사용해 데이터베이스 설정 과정을 생략함<br/>
만약 분산된 환경에서 예제를 수행하려면 다른 데이터베이스를 사용해야 함<br/>

*CAUTION*<br/>
SQLite가 설명용 예제로 적합하다고 해도 운영 환경에서 사용해서는 안 됨<br/>
또한 SQLite는 쓰기 연산 시 전체 데이터베이스에 락을 설정하므로 분산 환경에 적합하지 않음<br/>
이 장의 예제는 MySQL이나 PostgreSQL에서 실행해도 잘 동작함<br/>

데이터베이스의 데이터를 읽고 쓰기 위해서는 스파크 classpath에 데이터베이스의 JDBC(Java DataBase Connectivity) 드라이버를 추가하고 적절한 JDBC 드라이버 jar 파일을 제공해야 함<br/>
예를 들어 PostgreSQL 데이터베이스에 데이터를 읽거나 쓰려면 다음과 같이 실행함<br/>

In [29]:
/*
./bin/spark-shell \
--driver-class-path postgresql-9.4.1207.jar \
--jars postgresql-9.4.1207.jar
*/

다른 데이터 소스와 마찬가지로 SQL 데이터베이스에서 데이터를 읽고 쓸 때 사용할 수 있는 몇 가지 옵션이 있음<br/>
(자세한 내용은 책에 있는 표를 참고할 것)<br/>

JDBC 데이터소스 옵션<br/>
* url
* dbtable
* driver, partitionColumn, lowerBound, upperBound
* numPartitions
* fetchsize
* batchsize
* isolationLevel
* truncate
* createTableOptions
* createTableColumnTypes

## 9.6.1 SQL 데이터베이스 읽기

파일 읽기와 마찬가지로 SQL 데이터베이스에서 데이터를 읽는 방법은 다른 데이터 소스와 다르지 않음<br/>
다른 데이터 소스처럼 포맷과 옵션을 지정한 후 데이터를 읽어 들임<br/>

In [30]:
val driver = "org.sqlite.JDBC"
val path = "Downloads/Spark-The-Definitive-Guide/data/flight-data/jdbc/my-sqlite.db"
val url = s"jdbc:sqlite:/${path}"
val tablename = "flight_info"

driver: String = org.sqlite.JDBC
path: String = Downloads/Spark-The-Definitive-Guide/data/flight-data/jdbc/my-sqlite.db
url: String = jdbc:sqlite:/Downloads/Spark-The-Definitive-Guide/data/flight-data/jdbc/my-sqlite.db
tablename: String = flight_info


접속 관련 속성을 정의한 다음, 정상적으로 데이터베이스에 접속되는지 테스트해 해당 연결이 유효한지 확인 가능<br/>
이것은 스파크 드라이버가 데이터베이스에 접속할 수 있는지 확인할 수 있는 훌륭한 문제 해결 기술임<br/>
SQLite는 로컬 머신에 존재하는 파일 형태이므로 접속 테스트가 무의미할 수도 있음<br/>
하지만 MySQL 같은 데이터베이스를 사용하는 경우 다음과 같은 코드를 사용해 접속 테스트를 해볼 수 있음<br/>

In [31]:
import java.sql.DriverManager

val connection = DriverManager.getConnection(url)
connection.isClosed()
connection.close()

java.sql.SQLException:  No suitable driver found for jdbc:sqlite:/Downloads/Spark-The-Definitive-Guide/data/flight-data/jdbc/my-sqlite.db

*(위 부분 에러로 인해 9.6절 이하 예제는 진행할 수 없고 에러 고친 뒤 나중에 실행해볼 것)*

## 9.6.2 쿼리 푸시다운

### 데이터베이스 병렬로 읽기

### 슬라이딩 윈도우 기반의 파티셔닝

## 9.6.3 SQL 데이터베이스 쓰기

# 9.7 텍스트 파일

스파크는 일반 텍스트 파일(plain-text-file)도 읽을 수 있음<br/>
파일의 각 줄은 DataFrame의 레코드가 됨<br/>
그러므로 변환하는 것도 마음대로 할 수 있음<br/>
아파치 로그 파일을 구조화된 포맷으로 파싱하거나, 자연어 처리를 위해 일반 텍스트를 파싱하는 경우를 예로 들 수 있음<br/>
텍스트 파일은 기본 데이터 타입의 유연성을 활용할 수 있으므로 Dataset API에서 사용하기 매우 좋은 포맷임<br/>

## 9.7.1 텍스트 파일 읽기

텍스트 파일을 읽는 것은 매우 간단함<br/>
textFile 메서드에 텍스트 파일을 지정하기만 하면 됨<br/>
textFile 메서드는 파티션 수행 결과로 만들어진 디렉터리명을 무시함<br/>
파티션된 텍스트 파일을 읽거나 쓰려면 읽기 및 쓰기 시 파티션 수행 결과로 만들어진 디렉터리를 인식할 수 있도록 text 메서드를 사용해야 함<br/>

In [33]:
spark.read.textFile("Downloads/Spark-The-Definitive-Guide/data/flight-data/csv/2010-summary.csv")
    .selectExpr("split(value, ',') as rows").show(false)

+----------------------------------------------------+
|rows                                                |
+----------------------------------------------------+
|[DEST_COUNTRY_NAME, ORIGIN_COUNTRY_NAME, count]     |
|[United States, Romania, 1]                         |
|[United States, Ireland, 264]                       |
|[United States, India, 69]                          |
|[Egypt, United States, 24]                          |
|[Equatorial Guinea, United States, 1]               |
|[United States, Singapore, 25]                      |
|[United States, Grenada, 54]                        |
|[Costa Rica, United States, 477]                    |
|[Senegal, United States, 29]                        |
|[United States, Marshall Islands, 44]               |
|[Guyana, United States, 17]                         |
|[United States, Sint Maarten, 53]                   |
|[Malta, United States, 1]                           |
|[Bolivia, United States, 46]                        |
|[Anguilla

## 9.7.2 텍스트 파일 쓰기

# 9.8 고급 I/O 개념

쓰기 작업 전에 파티션 수를 조절함으로써 병렬로 처리할 파일 수를 제어할 수 있음<br/>
또한 **버켓팅**과 **파티셔닝**을 조절함으로써 데이터의 저장 구조를 제어할 수 있음<br/>
버켓팅과 파티셔닝은 잠시 후에 알아보겠음<br/>

## 9.8.1 분할 가능한 파일 타입과 압축 방식

특정 파일 포맷은 기본적으로 분할을 지원함<br/>
따라서 스파크에서 전체 파일이 아닌 쿼리에 필요한 부분만 읽을 수 있으므로 성능 향상에 도움이 됨<br/>
게다가 HDFS 같은 시스템을 사용한다면 분할된 파일을 여러 블록으로 나누어 분산 저장하기 때문에 훨씬 더 최적화할 수 있음<br/>
*이와 함께 압축 방식도 관리해야 함<br/>
모든 압축 방식이 분할 압축을 지원하지는 않음*<br/>
**데이터를 저장하는 방식에 따라 스파크 잡이 원활하게 동작하는 데 막대한 영향을 끼칠 수 있음<br/>
추천하는 파일 포맷과 압축 방식은 파케이 파일 포맷과 GZIP 압축 방식임**<br/>

## 9.8.2 병렬로 데이터 읽기

여러 executor가 같은 파일을 동시에 읽을 수는 없지만 여러 파일을 동시에 읽을 수는 있음<br/>
다수의 파일이 존재하는 폴더를 읽을 때 폴더의 개별 파일은 DataFrame의 파티션이 됨<br/>
따라서 사용 가능한 executor를 이용해 병렬(executor 수를 넘어가는 파일은 처리 중인 파일이 완료될 때까지 대기)로 파일을 읽음<br/>

## 9.8.3 병렬로 데이터 쓰기

파일이나 데이터 수는 데이터를 쓰는 시점에 DataFrame이 가진 파티션 수에 따라 달라질 수 있음<br/>
기본적으로 데이터 파티션당 하나의 파일이 작성됨<br/>
따라서 옵션이 지정된 파일명은 실제로는 다수의 파일을 가진 디렉터리임<br/>
그리고 디렉터리 안에 파티션당 하나의 파일로 데이터를 저장함<br/>

예를 들어 다음 코드는 폴더 안에 5개의 파일을 생성함<br/>

In [34]:
/*
csvFile.repartition(5).write.format("csv").save("/tmp/multiple.csv")
*/

### 파티셔닝

파티셔닝은 어떤 데이터를 어디에 저장할 것인지 제어할 수 있는 기능임<br/>
파티셔닝된 디렉터리 또는 테이블에 파일을 쓸 때 디렉터리별로 컬럼 데이터를 인코딩해 저장함<br/>
그러므로 데이터를 읽을 때 전체 데이터셋을 스캔하지 않고 필요한 컬럼의 데이터만 읽을 수 있음<br/>
이 방식은 모든 파일 기반의 데이터 소스에서 지원함<br/>

In [35]:
/*
csvFile.limit(10).write.mode("overwrite").partitionBy("DEST_COUNTRY_NAME")
    .save("/tmp/partitioned-files.parquet")
*/

파티셔닝은 필터링을 자주 사용하는 테이블을 가진 경우에 사용할 수 있는 가장 손쉬운 최적화 방식임<br/>
예를 들어 전체 데이터를 스캔하지 않고 지난주 데이터만 보려면 날짜를 기준으로 파티션을 만들 수 있음<br/>
이 기법을 사용하면 빠른 속도로 데이터를 읽어들일 수 있음<br/>

### 버켓팅

버켓팅(bucketing)은 각 파일에 저장된 데이터를 제어할 수 있는 또 다른 파일 조직화 기법임<br/>
*이 기법을 사용하면 동일한 버킷 ID를 가진 데이터가 하나의 물리적 파티션에 모두 모여 있기 때문에 데이터를 읽을 때 셔플을 피할 수 있음*<br/>
즉, 데이터가 이후의 사용 방식에 맞춰 사전에 파티셔닝되므로 조인이나 집계 시 발생하는 고비용의 셔플을 피할 수 있음<br/>

특정 컬럼을 파티셔닝하면 수억 개의 디렉터리를 만들어낼 수도 있음<br/>
이런 경우 데이터를 버켓팅할 수 있는 방법을 찾아야 함<br/>
다음은 '버켓' 단위로 데이터를 모아 일정 수의 파일로 저장하는 예제임<br/>

In [36]:
/*
val numberBuckets = 10
val columnToBucketBy = "count"

csvFile.write.format("parquet").mode("overwrite")
    .bucketBy(numberBuckets, columnToBucketBy).saveAsTable("bucketedFiles")
*/

버켓팅은 스파크 관리 테이블에서만 사용할 수 있음<br/>

## 9.8.4 복합 데이터 유형 쓰기

스파크는 다양한 자체 데이터 타입을 제공함<br/>
이러한 데이터 타입은 스파크에서는 잘 동작하지만 모든 데이터 파일 포맷에 적합한 것은 아님<br/>
예를 들어 CSV 파일은 복합 데이터 타입을 지원하지 않지만 파케이나 ORC는 복합 데이터 타입을 지원함<br/>

## 9.8.5 파일 크기 관리

파일 크기를 관리하는 것은 데이터를 저장할 때는 중요한 요소가 아님<br/>
하지만 데이터를 읽을 때는 중요한 요소 중 하나임<br/>
작은 파일을 많이 생성하면 메타데이터에 엄청난 관리 부하가 발생함<br/>
HDFS 같은 많은 파일 시스템은 작은 크기의 파일을 잘 다루지 못함<br/>
스파크는 특히 더 그렇고, 이런 상황을 '작은 크기의 파일 문제'라고 함<br/>
하지만 그 반대의 경우도 문제가 됨<br/>
몇 개의 로우가 필요하더라도 전체 데이터 블록을 읽어야 하기 때문에 비효율적임<br/>
그러므로 너무 큰 파일도 좋지 않음<br/>

스파크 2.2 버전에는 자동으로 파일 크기를 제어할 수 있는 새로운 방법이 도입되었음<br/>
이전 예제에서 결과 파일 수는 파일을 쓰는 시점(그리고 우리가 선택한 파티셔닝 컬럼)에서의 파티션 수에서 파생되었음을 알 수 있었음<br/>
이제 결과 파일을 최적의 크기로 제한할 수 있는 새로운 기능을 활용해보겠음<br/>
이 기능을 사용하려면 maxRecordsPerFile 옵션에 파일당 레코드 수를 지정해야 함<br/>
각 파일에 기록될 레코드 수를 조절할 수 있으므로 파일 크기를 더 효과적으로 제어할 수 있음<br/>
만약 파일 쓰기 객체에 다음과 같은 옵션을 설정했다면 스파크는 파일당 최대 5,000개의 로우를 포함하도록 보장할 수 있음<br/>

In [37]:
/*
df.write.option("maxRecordsPerFile", 5000)
*/

# 9.9 정리

이 장에서는 스파크에서 데이터를 읽고 쓸 때 사용할 수 있는 다양한 옵션을 알아보았음<br/>
이제 스파크를 사용하는 데 필요한 거의 모든 내용을 배웠음<br/>
흥미롭게도 사용자 정의 데이터 소스를 구현하는 방법도 있음<br/>
하지만 관련 API가 구조적 스트리밍과의 호환성을 위해 개선되고 있으므로 데이터 소스 구현 방법에 관한 내용은 생략했음<br/>
만약 사용자 정의 데이터 소스를 구현하는 방법을 알고 싶다면 모범 사례 중 하나인 카산드라 커넥터를 참고할 것<br/>

다음 장에서는 스파크 SQL에 대해 알아보겠음<br/>
그리고 지금까지 구조적 API를 사용했던 모든 것을 스파크 SQL로 실행하는 방법에 대해 자세히 알아보겠음<br/>